<a href="https://colab.research.google.com/github/AlkaidCheng/quple.github.io/blob/master/examples/Parameterized_Circuits_Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install quple

Import modules for construction of quantum circuits and parameterised quantum circuits

In [2]:
from quple import QuantumCircuit, ParameterisedCircuit

# Quantum Circuits

A quantum circuit is a system of quantum bits (called qubits) together with a sequence of unitary operations (quantum gates) applied to the qubits which transform the quantum state of the system. 

The construction of quantum circuits is implemented by the `QuantumCircuit` class which is a wrapper of the `cirq.Circuit` instance

Args:
* n_qubit (int, iterable of cirq.GridQubit)
    If int, it specifies the number of qubits in the circuit.
    If iterable of cirq.GridQubit object, it specifies the exact
    qubit layout of the circuit. 
* name: str
    Name of the circuit
* insert_strategy (cirq.InsertStrategy, default=None)
    The insertion strategy of gate operations in the circuit. If None, defaults to INLINE. 
* backend (default=None): The backend for the quantum circuit. If None, defaults to quantum simulator.

In [3]:
# Construction of the circuit for a Bell state

# Creates a circuit with 2 qubits
circuit = QuantumCircuit(2, name='BellCircuit')
# Apply the Hadamard gate to the qubit with index 0 
circuit.H(0)
# Apply the CNOT gate to qubits with indices 0 and 1
# with the first one being the control qubit
circuit.CNOT((0,1))
# Print out the circuit diagram
print(circuit)

(0, 0): ───H───@───
               │
(0, 1): ───────X───


In [4]:
# Construction of a circuit with custom qubit layout

import cirq
import numpy as np
qubits = [cirq.GridQubit(1,2), cirq.GridQubit(2, 3), cirq.GridQubit(3, 4)]

# Creates a circuit with the given qubits
circuit = QuantumCircuit(qubits)
# Apply the Hadamard gate to the qubits with indices 0, 1 and 2
circuit.H([0, 1, 2])
# Rotate the first qubit along x axis by an angle pi
circuit.RX(theta=np.pi, qubit_expr=0)
# Apply XX gate to the first and second qubit
circuit.XX((0, 1))
# Apply SWAP gate to the second and third qubit
circuit.SWAP((1,2))
# Print out the circuit diagram
print(circuit)

(1, 2): ───H───Rx(π)───XX───────
                       │
(2, 3): ───H───────────XX───×───
                            │
(3, 4): ───H────────────────×───


In [5]:
# Print out the number of qubits in the circuit
circuit.n_qubit

3

In [6]:
# Print out the qubits in the circuit
circuit.qubits

[cirq.GridQubit(1, 2), cirq.GridQubit(2, 3), cirq.GridQubit(3, 4)]

### Parameterizing a quantum circuit

A quantum circuit is said to be parameterized if it contains gate operations with undetermined parameter expressions. For example, the gate operation $Rx(\theta)$ is the Pauli rotation on the qubit about the $x$-axis by an angle parameterized by $\theta$. 

In [7]:
# Construction of a parameterised circuit

import sympy as sp
import numpy as np

# Creates a circuit with 3 qubits
circuit = QuantumCircuit(3, name='PQC')    
# Apply the Hadamard gate to all qubits
circuit.H(circuit.qubits)
# Create an array of symbols of size 3 with 'θ' as prefix
theta = sp.symarray('θ', 3)
# Apply the RZ gate to all qubits parameterised by θ
for i, qubit in enumerate(circuit.qubits):
  circuit.RZ(theta[i], qubit)
# Apply CNOT to qubits with indices 0 and 1 and
# qubits with indices 1 and 2 with the first one
# being the control qubit
circuit.CNOT([(0,1), (1, 2)])
# Print out the circuit diagram
print(circuit)

print('\n\n')
# Resolve the parameters of the circuit
parameter_values = np.array([np.pi, 2*np.pi, 3*np.pi])
resolved_circuit = circuit.resolve_parameters(parameter_values)
#Print out the resolved circuit
print(resolved_circuit)

(0, 0): ───H───Rz(θ_0)───@───────
                         │
(0, 1): ───H───Rz(θ_1)───X───@───
                             │
(0, 2): ───H───Rz(θ_2)───────X───



(0, 0): ───H───Rz(π)────@───────
                        │
(0, 1): ───H───Rz(2π)───X───@───
                            │
(0, 2): ───H───Rz(-π)───────X───


# Parameterized Quantum Circuits

In Quple, the construction of parameterized quantum circuits is implemented by the `ParameterizedCircuit` class based on the google Cirq library. It uses a specific quantum circuit design called the circuit-centric architecture which is composed of some $L$ copies (called the circuit depth) of a primary circuit block. 

For a quantum circuit with $n$ qubits, each circuit block consists of a layer (called the rotation layer) of single qubit gates applied to each qubit followed by a layer (called the entanglement layer) of two (or three) qubit gates to entangle the qubits with a given connectivity. It is a natural design that can be easily implemented in actual quantum computers and its strongly entangling nature has various advantages including error mitigation, the ability to efficiently represent the solution space of some machine learning tasks and to  capture nontrivial correlation in the quantum data.

Arguments:
* n_qubit (int): Number of qubits in the circuit
* copies (int): Number of times the layers are repeated (i.e. circuit depth).
* rotation_blocks: A list of single qubit gate operations to be applied in the rotation layer.
* entanglement_blocks: A list of multi qubit gate operations to be applied in the entanglement layer.
* entangle_strategy (default='full'): determines how the qubits are connected in an entanglement block.
* parameter_symbol (str): Symbol prefix for circuit parameters.
* final_rotation_layer (boolean): Whether to add an extra final rotation layer to the circuit.
* flatten_circuit (boolean): Whether to flatten circuit parameters when the circuit is modified.


In [8]:
# Construct a parameterised circuit with a rotation layer consisting of the 'H' and 'RZ' blocks, and an entanglement layer consisting of the 'CNOT' blocks with linear entanglement
pqc = ParameterisedCircuit(n_qubit=4, rotation_blocks=['H','RZ'], entanglement_blocks=['CNOT'], entangle_strategy='linear', copies=2)
# Print out the circuit diagram
print(pqc)

(0, 0): ───H───Rz(θ_0)───@───────H───Rz(θ_4)───@─────────────────
                         │                     │
(0, 1): ───H───Rz(θ_1)───X───@───H───Rz(θ_5)───X─────────@───────
                             │                           │
(0, 2): ───H───Rz(θ_2)───────X───@───H─────────Rz(θ_6)───X───@───
                                 │                           │
(0, 3): ───H───Rz(θ_3)───────────X───H─────────Rz(θ_7)───────X───


In [9]:
# Alternatively, the build() method can be called
pqc = ParameterisedCircuit(n_qubit=4)
pqc.build(rotation_blocks=['H','RZ'], entanglement_blocks=['CNOT'], entangle_strategy='linear', copies=2)
# Print out the circuit diagram
print(pqc)

(0, 0): ───H───Rz(θ_0)───@───────H───Rz(θ_4)───@─────────────────
                         │                     │
(0, 1): ───H───Rz(θ_1)───X───@───H───Rz(θ_5)───X─────────@───────
                             │                           │
(0, 2): ───H───Rz(θ_2)───────X───@───H─────────Rz(θ_6)───X───@───
                                 │                           │
(0, 3): ───H───Rz(θ_3)───────────X───H─────────Rz(θ_7)───────X───


In [10]:
# Alternatively, the rotation layer and entanglement layer can be added independently
pqc = ParameterisedCircuit(n_qubit=4)
pqc.add_rotation_layer(rotation_blocks=['H','RZ'])
pqc.add_entanglement_layer(entanglement_blocks=['CNOT'], entangle_strategy='linear')
# Print out the circuit diagram
print(pqc)

(0, 0): ───H───Rz(θ_0)───@───────────
                         │
(0, 1): ───H───Rz(θ_1)───X───@───────
                             │
(0, 2): ───H───Rz(θ_2)───────X───@───
                                 │
(0, 3): ───H───Rz(θ_3)───────────X───


In [11]:
# Print out the parameter symbols in the circuit
pqc.parameters

array([θ_0, θ_1, θ_2, θ_3], dtype=object)

In [12]:
# Print out the number of parameters in the circuit
pqc.num_param

4